In [1]:
import os
import sys
import  time
import random
from itertools import chain
import numpy as np
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
plt.style.use("ggplot")

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Dropout, Dense
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.metrics import Recall, Precision

from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from skimage.io import imread, imshow

import h5py

In [2]:
im_width = 128
im_height = 128
n_epochs = 10
batch_size = 32
num_class = 2

def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x


def get_unet(input_img, n_filters = 64, dropout = 0.2, batchnorm = True):
    """Function to define the UNET Model"""
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = Conv2D(num_class, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model


def TV_bin_loss(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    bin_loss = binary_crossentropy(y_true_f, y_pred_f)
    images = y_pred[: : ,: ,1]
    value = tf.reduce_mean(tf.abs(images[:,1:,:] - images[:,:-1,:])) + tf.reduce_mean(tf.abs(images[:,:,1:] - images[:,:,:-1]))
    return 2.4e-7*value + bin_loss


def dice_coef(y_pred, y_true):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 0.0001) / (K.sum(y_true_f) + K.sum(y_pred_f) + 0.0001)


def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


def custom_loss(y_true, y_pred):
    layer_names=[layer.name for layer in model.layers]
    for l in layer_names:
        if l==layer_names[-1]:
            value = TV_bin_loss(y_true, y_pred)
        else:
            value = binary_crossentropy(K.flatten(y_true),K.flatten(y_pred))
    return value

In [3]:
combined_data = h5py.File("h5_datasets/combined_CT_datasets.h5", "r")

X_train = np.array(combined_data["X_train"])
X_valid = np.array(combined_data["X_valid"])
y_train = np.array(combined_data["y_train"])
y_valid = np.array(combined_data["y_valid"])

In [4]:
input_img = Input((im_height, im_width, 1), name='img')
model = get_unet(input_img, n_filters=64, dropout=0.2, batchnorm=True)
model.compile(optimizer=Adam(learning_rate=0.001), loss=[custom_loss], metrics=['accuracy', dice_loss, Recall(name='recall_1'), Precision(name='pre_1')])

callbacks = [
    EarlyStopping(patience=50, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-TV-UNet1.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

results = model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs, callbacks=callbacks, validation_data=(X_valid, y_valid))

2022-03-18 12:05:37.891179: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-18 12:05:37.905135: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-18 12:05:37.906786: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-18 12:05:37.908956: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/5


2022-03-18 12:05:53.717015: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2022-03-18 12:05:54.760742: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


41/41 [==============================] - ETA: 0s - loss: 0.3906 - accuracy: 0.8812 - dice_loss: 0.2785 - recall_1: 0.9313 - pre_1: 0.8410
Epoch 00001: val_loss improved from inf to 11.03044, saving model to model-TV-UNet1.h5
41/41 [==============================] - 54s 819ms/step - loss: 0.3906 - accuracy: 0.8812 - dice_loss: 0.2785 - recall_1: 0.9313 - pre_1: 0.8410 - val_loss: 11.0304 - val_accuracy: 0.3626 - val_dice_loss: 0.7502 - val_recall_1: 0.2411 - val_pre_1: 0.2614 - lr: 0.0010
Epoch 2/5
41/41 [==============================] - ETA: 0s - loss: 0.1994 - accuracy: 0.9616 - dice_loss: 0.1502 - recall_1: 0.9616 - pre_1: 0.9615
Epoch 00002: val_loss improved from 11.03044 to 0.61745, saving model to model-TV-UNet1.h5
41/41 [==============================] - 21s 522ms/step - loss: 0.1994 - accuracy: 0.9616 - dice_loss: 0.1502 - recall_1: 0.9616 - pre_1: 0.9615 - val_loss: 0.6174 - val_accuracy: 0.9488 - val_dice_loss: 0.0771 - val_recall_1: 0.9344 - val_pre_1: 0.9516 - lr: 0.0010
E

In [5]:
## save model as ONNX

if not os.path.exists('tf_model'):
    os.makedirs('tf_model')

if not os.path.exists('onnx_trained_model'):
    os.makedirs('onnx_trained_model')

tf.saved_model.save(model, 'tf_model')

2022-03-18 12:08:05.908431: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: tf_model/assets


In [6]:
%%bash

python -m tf2onnx.convert --saved-model tf_model --output onnx_trained_model/ct_seg_model.onnx --opset 14

/opt/conda/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-03-18 12:08:18,746 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2022-03-18 12:08:34,577 - INFO - Signatures found in model: [serving_default].
2022-03-18 12:08:34,578 - WARNING - '--signature_def' not specified, using first signature: serving_default
2022-03-18 12:08:34,578 - INFO - Output names: ['conv2d_18']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2022-03-18 12:08:39,549 - WARNING - From /opt/conda/lib/python3.9/site-packages/tf2onnx/tf_loader.py:706: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2022-03-18 12:08:42,218 - INFO - U

In [7]:
# Release GPU memory
from numba import cuda 
cuda.select_device(0)
cuda.close()